# digitraffic.fi API-datan käsittelyä pythonilla

Liikenneviraston tieliikenteen mittauspisteiden dataa on vapaasti saatavilla verkossa.
Tässä koodiesimerkissä luemme tätä dataa ja muokkaamme siitä GeoJSON-formaatin mukaisen mittauspisteluettelon, joka voidaan tulostaa kartan yhteydessä (openstreetmap). Anturidataa on tarjolla hyvinkin paljon, mutta käytämme siitä vain ilman lämpötiloja.

In [1]:
from io import BytesIO
import gzip
from urllib.request import Request, urlopen

# apurutiini joka ilmoittaa www-palvelimelle, että voimme vastaanottaa gzip-pakattua dataa (Reqeust)
# jos data tulee tässä muodossa (response), niin käytämme moduulia gzip sen purkamiseen
def ReadDataURL( url ):
    req = Request( url )
    req.add_header( 'Accept-encoding', 'gzip' )
    response = urlopen( req )
    if response.info().get('Content-Encoding') == 'gzip':
        buffer = BytesIO( response.read() )
        with gzip.GzipFile( fileobj=buffer ) as MemoryFile:
            # gzip-pakattu data palautetaan purettuna:
            return MemoryFile.read()
    else:
        # "tavallinen" data palautetaan sellaisenaan:
        return response.read()
    

In [2]:
import json

# metadata sisältää asemien sijaintitiedot ja nimet
metaRaw = ReadDataURL( 'https://tie.digitraffic.fi/api/v1/metadata/weather-stations?lastUpdated=false' )

# verkosta tuleva data on JSON-formaatissa.
# muutetaan se Pythonin dictionary rakenteeksi:
metaData = json.loads( metaRaw.decode('utf-8') )

# yhden aseman tiedot tulostettuna
print( json.dumps( metaData['features'][0], indent=2 ) )

{
  "id": 1001,
  "geometry": {
    "coordinates": [
      24.59641241278282,
      60.228047274401014,
      0.0
    ],
    "type": "Point"
  },
  "properties": {
    "master": true,
    "state": "OK",
    "liviId": null,
    "municipality": "Espoo",
    "country": null,
    "municipalityCode": "49",
    "name": "vt1_Nupuri_1",
    "provinceCode": "1",
    "coordinatesETRS89": [
      366877.0,
      6679233.0,
      0.0
    ],
    "province": "Uusimaa",
    "collectionInterval": 300,
    "startTime": "2009-10-30T00:00:00+02:00",
    "repairMaintenanceTime": "2017-02-21T11:25:45+02:00",
    "roadStationId": 1001,
    "collectionStatus": "GATHERING",
    "annualMaintenanceTime": "2016-09-26T00:00:00+03:00",
    "stationSensors": [
      1,
      2,
      3,
      4,
      5,
      6,
      7,
      8,
      9,
      10,
      11,
      16,
      17,
      18,
      21,
      22,
      23,
      24,
      25,
      26,
      27,
      28,
      29,
      30,
      31,
      32,
      33

In [3]:
# otetaan talteen kunkin aseman tunnisteella sen koordinaatit ja nimi
STATIONS = dict()
for station in metaData['features']:
    # tämä 'geometry'  ja 'properties' rakenne on sama kuin GeoJSON-formaatissa:
    STATIONS[ station['id'] ] = { 'geometry'  : station['geometry'], 
                                  'properties': {'name' : station['properties']['name'] }
                                }
    
STATIONS

{1001: {'geometry': {'coordinates': [24.59641241278282,
    60.228047274401014,
    0.0],
   'type': 'Point'},
  'properties': {'name': 'vt1_Nupuri_1'}},
 1002: {'geometry': {'coordinates': [24.88969200565967,
    60.165869238798045,
    0.0],
   'type': 'Point'},
  'properties': {'name': 'kt51_Hki_Lapinlahti_R'}},
 1003: {'geometry': {'coordinates': [24.236497426453298,
    60.31219825239905,
    0.0],
   'type': 'Point'},
  'properties': {'name': 'st110_Myllylampi_R'}},
 1004: {'geometry': {'coordinates': [25.065093117308795,
    60.25183296052867,
    0.0],
   'type': 'Point'},
  'properties': {'name': 'vt4_Jakomäki_R'}},
 1005: {'geometry': {'coordinates': [25.124370867738254,
    60.42627352290708,
    0.0],
   'type': 'Point'},
  'properties': {'name': 'vt4_Kerava_1'}},
 1006: {'geometry': {'coordinates': [25.9139602081798, 60.4904970801801, 0.0],
   'type': 'Point'},
  'properties': {'name': 'vt7_Koskenkylä_R'}},
 1007: {'geometry': {'coordinates': [23.892507755546706,
    60.63

In [4]:
# ajankohtainen anturidata kaikilta asemilta löytyy tästä URL:sta:
weatherRaw = ReadDataURL('https://tie.digitraffic.fi/api/v1/data/weather-data?lastUpdated=false')
weatherData = json.loads( weatherRaw.decode('utf-8') )

print( json.dumps( weatherData['weatherStations'][0], indent=2) ) # testitulostus yhdestä asemasta

{
  "id": 2048,
  "measuredTime": "2017-03-14T15:00:00+02:00",
  "sensorValues": [
    {
      "shortName": "Ilma ",
      "roadStationId": 2048,
      "sensorValue": 2.5,
      "oldName": "airtemperature1",
      "id": 1,
      "name": "ILMA",
      "sensorUnit": "\u00b0C"
    },
    {
      "shortName": "DIlm",
      "roadStationId": 2048,
      "sensorValue": 0.2,
      "oldName": "airtemperature1change",
      "id": 2,
      "name": "ILMA_DERIVAATTA",
      "sensorUnit": "\u00b0C/h"
    },
    {
      "shortName": "Tie1 ",
      "roadStationId": 2048,
      "sensorValue": 3.8,
      "oldName": "roadsurfacetemperature1",
      "id": 3,
      "name": "TIE_1",
      "sensorUnit": "\u00b0C"
    },
    {
      "shortName": "DTie1",
      "roadStationId": 2048,
      "sensorValue": 0.2,
      "oldName": "roadsurfacetemperature1change",
      "id": 4,
      "name": "TIE_1_DERIVAATTA",
      "sensorUnit": "\u00b0C/h"
    },
    {
      "shortName": "Tie2",
      "roadStationId": 2048,
    

In [5]:
# poimitaan jokaiselta asemalta talteen "ILMA" anturin arvo
for station in weatherData['weatherStations']:
    for sensor in station['sensorValues']:
        if sensor['name'] == 'ILMA':
            # STATIONS-rakenne on aiemmin jo luotu asemien tiedoista
            # tässä lisäämme yhden 'properties' kentän lämpötilaa varten:
            STATIONS[sensor['roadStationId']]['properties']['Temperature'] = sensor['sensorValue']

# kaikki kerätty data
STATIONS            

{1001: {'geometry': {'coordinates': [24.59641241278282,
    60.228047274401014,
    0.0],
   'type': 'Point'},
  'properties': {'Temperature': 1.7, 'name': 'vt1_Nupuri_1'}},
 1002: {'geometry': {'coordinates': [24.88969200565967,
    60.165869238798045,
    0.0],
   'type': 'Point'},
  'properties': {'Temperature': 1.1, 'name': 'kt51_Hki_Lapinlahti_R'}},
 1003: {'geometry': {'coordinates': [24.236497426453298,
    60.31219825239905,
    0.0],
   'type': 'Point'},
  'properties': {'Temperature': 1.2, 'name': 'st110_Myllylampi_R'}},
 1004: {'geometry': {'coordinates': [25.065093117308795,
    60.25183296052867,
    0.0],
   'type': 'Point'},
  'properties': {'Temperature': 1.3, 'name': 'vt4_Jakomäki_R'}},
 1005: {'geometry': {'coordinates': [25.124370867738254,
    60.42627352290708,
    0.0],
   'type': 'Point'},
  'properties': {'Temperature': 1.3, 'name': 'vt4_Kerava_1'}},
 1006: {'geometry': {'coordinates': [25.9139602081798, 60.4904970801801, 0.0],
   'type': 'Point'},
  'properties

In [6]:
# luodaan dictionary, joka sisältää asemien tiedot GeoJSON-formaatissa
# https://en.wikipedia.org/wiki/GeoJSON
GEO = dict()
GEO['type'] = 'FeatureCollection'
GEO['features'] = list() # tässä listassa on kaikki "pisteet" eli mittausasemat

for station in STATIONS.keys():
    # aiemmin kerätty data on jo oikeassa muodossa GeoJSON:n kannalta,
    # joten tyyppitiedon lisäämisen jälkeen
    # talletamme vain kunkin aseman 'features' listaan
    ITEM = STATIONS[station]
    ITEM['type'] = 'Feature'
    GEO['features'].append( ITEM )
   
GEO

{'features': [{'geometry': {'coordinates': [23.106233758256906,
     60.41739531884564,
     0.0],
    'type': 'Point'},
   'properties': {'Temperature': 2.5, 'name': 'vt1_Halikko_O'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [23.211233122503284, 60.407402202207926, 0.0],
    'type': 'Point'},
   'properties': {'Temperature': 2.2, 'name': 'vt1_Haukkala_R'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [23.211233122503284, 60.407402202207926, 0.0],
    'type': 'Point'},
   'properties': {'Temperature': 2.2, 'name': 'vt1_Haukkala_O'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [24.247808636106395, 63.03732129068302, 0.0],
    'type': 'Point'},
   'properties': {'Temperature': 0.5, 'name': 'vt16_Möksy_Opt'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [19.62046834353343, 60.20849907380442, 0.0],
    'type': 'Point'},
   'properties': {'Temperature': 3.1, 'name': 'Åland_Eckerö_K'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [24.03270009

In [7]:
# jupyter notebookin lisäosien avulla tehdään tekstikenttä (label)
# ja kartta (map)
from ipyleaflet import Map, GeoJSON
import ipywidgets as ipyw
label = ipyw.Label(layout=ipyw.Layout(width='100%'))
label.value = u'0.0'

# kartan päälle luodaan GeoJSON pisteitä, jotka vastaavat edellä kerättyjä sääasemia
layer = GeoJSON( data = GEO )

# kun karttapistettä klikataan, niin tulostetaan tekstikenttään GeoJSON properties -tietoihin 
# talletettuna oleva lämpötila ja aseman nimi
def click_handler(event=None,id=None,properties=None):
    label.value = str( properties.get('Temperature') ) +' ℃ : ' + properties.get('name')

layer.on_click(click_handler)

# luodaan kartta keskitettynä Tampereelle
map = Map( zoom=7, center=[61.4978,23.7610])

# liitetään kartan päälle sääasemien pisteet
map.add_layer( layer )

# näytetään tekstikenttä ja kartta tässä notebook:ssa
ipyw.VBox( [label,map] )

Widget Javascript not detected.  It may not be installed or enabled properly.
